In [1]:
import numpy as np

In [6]:
# %% Sanity check for capital Theta
def sanityCheckTheta(D,Rstar,theta):
    """
    Do the sanity check for capital Theta that the inputs satisfy
    -1 <= d*sin(theta)/D <= 1 where d is the l.o.s distance
    
    Input
    ------
    D: the SNv propagation distance D
    Rstar: the distance between Earth and SN
    theta: the open-angle in rad
    
    Output
    ------
    bool: is the inputs sane?
    """
    # Get l.o.s d
    d = d_los(D,Rstar,theta)
    return -1 <= d*np.sin(theta)/D <= 1


# %% Sanity check for alpha
def sanityCheckAlpha(D,Rstar,theta):
    """
    Do the sanity check for scattering angle alpha that the inputs
    satisfy -1 <= Rstar*sin(theta)/D <= 1
    
    Input
    ------
    D: the SNv propagation distance D
    Rstar: the distance between Earth and SN
    theta: the open-angle in rad
    
    Output
    ------
    bool: is the inputs sane?
    """
    return -1 <= Rstar*np.sin(theta)/D <= 1
    

# %% Get the scattering angle alpha
def getAlpha(D,Rstar,theta):
    """
    Get the scattering angle alpha via law of cosine.
    If we did it with law of cosine, then for the case of alpha > 0.5pi,
    it will always return pi - alpha which cannot reflect the pratical
    situation
    
    Input
    ------
    D: the SNv propagation distance D
    Rstar: the distance between Earth and SN
    theta: the open-angle in rad
    
    Output
    ------
    alpha: scattering angle alpha in rad
    """
    # Find l.o.s. d^2
    d2 = d_los(D,Rstar,theta,True)
    d = np.sqrt(d2)
    # Get cos(alpha)
    cosAlpha = (Rstar**2 - D**2 - d2)/(2*D*d)
    # Get alpha via arccos
    alpha = np.arccos(cosAlpha)
    return alpha


# %% Calculate the l.o.s distance d
def d_los(D,Rstar,theta,is_square = False):
    """
    Calculate the l.o.s distance d 
    
    Input
    ------
    D: the SNv propagation distance D
    Rstar: the distance between Earth and SN
    theta: the open-angle in rad
    is_square: return the square of such distance, default is False
    
    Output
    ------
    d: the l.o.s distance d
    """
    sinTheta = np.sin(theta)
    common = D**2 + Rstar**2 - 2*Rstar**2*sinTheta**2
    root = 2*Rstar*np.sqrt((D + Rstar*sinTheta)*(D - Rstar*sinTheta)*(1 - sinTheta)*(1 + sinTheta))
    
    # d square, there are two solutions, one with minus root the other plus root
    # Since the input theta is always acute, we have to check the d we picked up also corresponds
    # to the theta that is acute. If the corresponds theta is obtuse, we have to change sign
    # Firstly, start with minus root
    d2 = common - root
    # Check the corresponding theta is acute (0<=cos(theta)<=1) or obtuse (-1<=cos(theta)<0)
    cosTheta = (Rstar**2 + d2 - D**2)/(2*Rstar*np.sqrt(d2))
    if 0 <= cosTheta <= 1:
        # theta is acute as desired
        pass
    else:
        # theta is obtuse and we have to change to plus root
        d2 = common + root
    
    # Return the correct l.o.s d according to user's choice    
    if is_square is False:
        return np.sqrt(d2)
    else:
        return d2
    
    
# %% Calculate ell
def ell(Rstar,Re,theta,beta,is_square = False):
    """
    Calculate the distance ell
    
    Input
    ------
    Rstar: the distance between Earth and SN
    Re: the distance between Earth and the GC
    theta: the open-angle in rad
    beta: the off-center angle in rad
    is_square: return the square of such distance, default is False
    
    Output
    ------
    ell: the distance ell
    """
    # Calculate d^2
    d2 = d_los(D,Rstar,theta,True)
    # Get d
    small_d = np.sqrt(d2)
    # Calculate ell^2
    ell2 = Re**2 + d2*np.cos(theta)**2 - 2*Re*small_d*np.cos(theta)*np.cos(beta)
    if is_square is False:
        return np.sqrt(ell2)
    else:
        return ell2


# %% Calculate r'
def rPrime(D,Rstar,Re,theta,phi,beta,tolerance = 1e-15):
    """
    Calculate the distance from boosted point to GC r'
    
    Input
    ------
    d: the l.o.s distance d
    Rstar: the distance between Earth and SN
    Re: the distance between Earth and the GC
    theta: the open-angle in rad
    phi: the azimuth angle in rad
    beta: the off-center angle in rad
    
    Output
    ------
    r': the distance r'
    """
    # ell^2
    ell2 = ell(Rstar,Re,theta,beta,True)
    # d^2
    d2 = d_los(D,Rstar,theta,True)
    # h
    h = np.sqrt(d2)*np.sin(theta)
    # cos(iota) and iota
    cosIota = (Re**2 - ell2 - d2*np.cos(theta)**2)/(2*np.cos(theta)*np.sqrt(ell2*d2))
    # Using sin(arccos(x)) = sqrt(1-x^2)
    if 0 <= np.abs(cosIota) <= 1:
        # normal case
        sinIota = np.sqrt(1 - cosIota**2)
    elif np.abs(cosIota) - 1 < tolerance:
        # cosIota is outside the valid range but its value is still within the
        # tolerance, which implies abs(cosIota) = 1 is still ok to our calculation.
        # Since cosIota is slight larger than 1 but within the tolerance, we switch 1 - x^2 to x^2 - 1
        # to avoid minus sign occuring in square root
        sinIota = np.sqrt(cosIota**2 - 1)
    else:
        # cosIota is not only outside the valid range but also untolerable
        raise print('cosIota value is outside the tolarance range, please check again')
    # r'^2
    rp2 = ell2*cosIota**2 + (np.sqrt(ell2)*sinIota - h*np.sin(phi))**2 + h**2*np.cos(phi)**2
    return np.sqrt(rp2)


In [18]:
# %% Get the scattering angle alpha
def get_alpha(d,Rstar,theta):
    """
    Get the scattering angle alpha via law of cosine.
    If we did it with law of cosine, then for the case of alpha > 0.5pi,
    it will always return pi - alpha which cannot reflect the pratical
    situation
    
    Input
    ------
    d: the l.o.s distance d
    Rstar: the distance between Earth and SN
    theta: the open-angle in rad
    
    Output
    ------
    alpha: scattering angle alpha in rad
    """
    # Get D^2
    D2 = get_D(d,Rstar,theta,True)
    D = np.sqrt(D2)
    # Get cos(alpha)
    cosAlpha = (Rstar**2 - D2 - d**2)/(2*D*d)
    # Get alpha via arccos
    alpha = np.arccos(cosAlpha)
    return alpha


# %% Calculate D
def get_D(d,Rstar,theta,is_square = False):
    """
    Calculate the distance between SN and boosted point D
    
    Input
    ------
    d: the l.o.s distance d
    Rstar: the distance between Earth and SN
    theta: the open-angle in rad
    is_square: return the square of such distance, default is False
    
    Output
    ------
    D: the distance D
    """
    # Calculate D^2 via law of cosine
    D2 = d**2 + Rstar**2 - 2*d*Rstar*np.cos(theta)
    if is_square is False:
        return np.sqrt(D2)
    elif is_square is True:
        return D2
    else:
        raise ValueError('\'is_square\' must be either True or False')
    

# %% Calculate ell
def get_ell(d,Re,theta,beta,is_square = False):
    """
    Calculate the distance ell
    
    Input
    ------
    d: the l.o.s distance d
    Re: the distance between Earth and the GC
    theta: the open-angle in rad
    beta: the off-center angle in rad
    is_square: return the square of such distance, default is False
    
    Output
    ------
    ell: the distance ell
    """
    # Calculate ell^2 via law of cosine
    ell2 = Re**2 + (d*np.cos(theta))**2 - 2*Re*d*np.cos(theta)*np.cos(beta)
    if is_square is False:
        return np.sqrt(ell2)
    elif is_square is True:
        return ell2
    else:
        raise ValueError('\'is_square\' must be either True or False')


# %% Calculate r'
def get_rprime(d,Rstar,Re,theta,phi,beta,tolerance = 1e-15):
    """
    Calculate the distance from boosted point to GC r'
    
    Input
    ------
    d: the l.o.s distance d
    Rstar: the distance between Earth and SN
    Re: the distance between Earth and the GC
    theta: the open-angle in rad
    phi: the azimuth angle in rad
    beta: the off-center angle in rad
    
    Output
    ------
    r': the distance r'
    """
    # ell^2
    ell2 = get_ell(d,Re,theta,beta,True)
    # D^2
    D2 = get_D(d,Rstar,theta,True)
    # h
    h = d*np.sin(theta)
    # cos(iota) and iota
    cosIota = (Re**2 - ell2 - (d*np.cos(theta))**2)/(2*np.cos(theta)*np.sqrt(ell2)*d)
    # Using sin(arccos(x)) = sqrt(1-x^2)
    if 0 <= np.abs(cosIota) <= 1:
        # normal case
        sinIota = np.sqrt(1 - cosIota**2)
    elif np.abs(cosIota) - 1 < tolerance:
        # cosIota is outside the valid range but its value is still within the
        # tolerance, which implies abs(cosIota) = 1 is still ok to our calculation.
        # Since cosIota is slight larger than 1 but within the tolerance, we switch 1 - x^2 to x^2 - 1
        # to avoid minus sign occuring in square root
        sinIota = np.sqrt(cosIota**2 - 1)
    else:
        # cosIota is not only outside the valid range but also untolerable
        raise print('cosIota value is outside the tolarance range, please check again')
    # r'^2
    rp2 = ell2*cosIota**2 + (np.sqrt(ell2)*sinIota - h*np.sin(phi))**2 + h**2*np.cos(phi)**2
    return np.sqrt(rp2)


# Calculate l.o.s d for a given time
def get_d

In [19]:
D = 11
Re = 8.5
Rstar = 18
theta = 0.01*np.pi
phi = np.pi/4
beta = 0.045

In [16]:
from scipy.integrate import quad

In [25]:
quad(lambda phi_x: rPrime(D,Rstar,Re,theta,phi_x,beta),0,2*np.pi)

(40.92592245672427, 8.617053479670056e-12)

In [14]:
rPrime(D,Rstar,Re,theta,phi,beta)

1.5143760438933807

In [20]:
get_rprime(d_los(D,Rstar,theta),Rstar,Re,theta,phi,beta,tolerance = 1e-15)

1.5143760438933807

In [15]:
print(d_los(D,Rstar,theta),
sanityCheckTheta(D,Rstar,theta),
sanityCheckAlpha(D,Rstar,theta),
ell(Rstar,Re,theta,beta),
(Re- d_los(D,Rstar,theta)*np.cos(theta)),
rPrime(D,Rstar,Re,theta,phi,beta))

7.005658150586865 True True 1.5374999363678117 1.4977987188087702 1.5143760438933807


In [36]:
getAlpha(D,Rstar,theta)/np.pi

0.028366203012016263

In [37]:
sanityCheckAlpha(D,Rstar,theta)

True

In [40]:
np.cos(theta)

0.9995065603657316

In [46]:
np.arccos((-D**2 + Rstar**2 + d_los(D,Rstar,theta,True))/(2*Rstar*d_los(D,Rstar,theta)))

3.110176727053904

In [44]:
d_los(D,Rstar,theta)

2.5009535579126934

In [48]:
np.pi-theta

3.1101767270538954